<a href="https://colab.research.google.com/github/gmauricio-toledo/SA-paper/blob/master/SMS-experiments-borrar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar los archivos del proyecto

In [ ]:
!python -m spacy download es_core_news_sm

## Opción 1: Traer sólo los módulos con las clases:

In [11]:
import requests

def get_module(url):
    fname = url.split('/')[-1]
    r = requests.get(url)
    with open(fname, 'w') as f:
        f.write(r.text)

In [ ]:
urls = ["https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/experiment_tools_SA.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/scoring.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/SentimentKW.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/TextCleaner.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/TextRank.py"
        ]

for url in urls:
    get_module(url)

## Opción 2: Traer todo el repositorio github en el root

In [1]:
!git init
!git pull https://github.com/gmauricio-toledo/SA-paper.git 

Initialized empty Git repository in /content/.git/


# Experiments

In [6]:
from gensim.models import Word2Vec
from scoring import Scoring
import pandas as pd
import numpy as np

## Single run

In [8]:
import pandas as pd
from gensim.models import Word2Vec

w2v_model = Word2Vec.load("data/word2vec.model")

# ---- Leemos los mensajes ----
msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
# msjs_df['Normalized Label'] = (2/5)*msjs_df['ds'].values-1
# y = msjs_df['ds'].values

msjs_df.sample(3)

,Llave,Paciente,Sexo,cal_vida,Texto,ds03,ds45,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5,ds,dp
105,556,124,1,35.4,día entretenido sigo alguna molestia estomago ...,1,0,0,1.0,0,0,0,1,0,0,3,1.0
2252,12203,4886,1,90.0,conseguí entrevistarme alguien interesa remiti...,0,1,1,0.0,0,0,0,0,0,1,5,0.0
4725,17949,11242,2,0.0,día febrero pues miso mañana tranquilo habitac...,0,1,0,1.0,0,0,0,0,1,0,4,1.0


In [9]:
from experiment_tools_SA import SentimentAnalysis


# betas1 = [0.5,1,2]#np.linspace(0.1,2,5)
# betas2 = [0.5,1,2]#np.linspace(0.5,5,5)
# nums_cols = [3,5,7]
# alphas = [0.5,0.75,0.9]#np.linspace(0.25,0.98,5)

# params_dict = {
#                 # 'beta1':betas1,
#                 # 'beta2':betas2,
#                 'n_cols':nums_cols
#                 # 'alpha':alphas
#                 }



hpd = {'emb_model': w2v_model}

sa = SentimentAnalysis(hyper_params_dict=hpd,
                        df=msjs_df,
                        text_col_name='Texto',
                        label_col_name='ds',
                        )

Dataframe contains entries with 6 labels:
[0 1 2 3 4 5]


In [ ]:
comb_dict = {
            'beta1': 1,
            'beta2': 5,
            'n_cols': 9,
            'alpha': 0.9,
            'top_n': 100,
            'n_iter': 3
            }

results = sa.run(combination_dict=comb_dict)

Calculando palabras prototípicas... 

In [ ]:
print(results)

{'test_loss': 1.2134568691253662, 'test_accuracy': 0.53969407081604, 'predictions': array([3, 4, 5, ..., 5, 5, 0])}


## Gridsearch

In [ ]:
from experiment_tools_SA import SentimentAnalysis
import pandas as pd
from gensim.models import Word2Vec

w2v_model = Word2Vec.load("data/word2vec.model")

# ---- Leemos los mensajes ----
msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
# msjs_df['Normalized Label'] = (2/5)*msjs_df['ds'].values-1
# y = msjs_df['ds'].values

display(msjs_df.sample(3))

hpd = {'emb_model': w2v_model}

sa = SentimentAnalysis(hyper_params_dict=hpd,
                        df=msjs_df,
                        text_col_name='Texto',
                        label_col_name='ds',
                        )

,Llave,Paciente,Sexo,cal_vida,Texto,ds03,ds45,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5,ds,dp
214,898,182,2,44.6,nota apuntada si hijo consigue quiere quiere f...,1,0,0,1.0,0,1,0,0,0,0,1,1.0
2510,12933,5230,1,21.4,levanto cansada bastantes dolores espalda braz...,0,1,1,0.0,0,0,0,0,0,1,5,0.0
1119,7105,2305,2,2.2,cansado torpe hoy caído calle puede ser cada v...,1,0,0,1.0,1,0,0,0,0,0,0,1.0


Dataframe contains entries with 6 labels:
[0 1 2 3 4 5]


In [ ]:
betas1 = [0.5,1]#np.linspace(0.1,2,5)
betas2 = [1,2]#np.linspace(0.5,5,5)
nums_cols = [3,7]
alphas = [0.9]#np.linspace(0.25,0.98,5)

params_dict = {
                'beta1':betas1,
                'beta2':betas2,
                'n_cols':nums_cols,
                'alpha':alphas
                }


dpd = {
                'beta1': 1,
                'beta2': 1,
                'n_cols': 5,
                'alpha': 0.5,
                'top_n': 50,
                'n_iter': 3
                }

sa.grid_search(param_dict=params_dict,
                default_params_dict=dpd)